# Описание задачи

В некоторой архитектуре нейронной сети зависимость выхода $y \in \mathbb{R}$ от входа $(x_1, x_2) \in \mathbb{R}^2$ можно записать следующим образом:
$$
\begin{align*}
&f_1(x_1, x_2) = x_1 + x_2,\\
&f_2(x_1, x_2) = x_1 \cdot x_2,\\
&g_1(x_1, x_2) = \tan( f_1(x_1, x_2) + f_2(x_1, x_2) + 100),\\
&g_2(x_1, x_2) = f_1(x_1, x_2) \cdot f_2(x_1, x_2),\\
&y(x_1, x_2) = p(g_1(x_1,x_2), g_2(x_1, x_2)),
\end{align*}
$$
где $p(g_1, g_2)-$ некоторая неизвестная функция.

С помощью [механизма автоматического дифференцирования в PyTorch](https://pytorch.org/tutorials/beginner/blitz/autograd_tutorial.html#) необходимо вычислить значения частных производных в разных точках:
$$
\frac{\partial y}{\partial x_1}, \frac{\partial y}{\partial x_2}, 
$$

Для этого в файле `dev.json` приведены значения $\frac{\partial p}{\partial g_1}, \frac{\partial p}{\partial g_2}$ (обозначены как `dpdg1, dpdg2`), вычисленные при известных значениях входа. 

Нужно:

1. Загрузить данные из файла `dev.json`
2. Посчитать значения нужных производных для каждой точки.
3. Сохранить результат в формате CSV. Файл должен иметь следующую структуру:
```
id,dx1,dx2
....
```
`id` - это значение `id` из `dev.json`, `dx1, dx2`- значение производных, вычисленные в точке `id` из `dev.json.`

Ответ проверяется с помощью функции [math.isclose](https://docs.python.org/3/library/math.html#math.isclose) с точностью до `1e-8`. 

# Решение
$
\begin{align*}&
\frac{\partial y}{\partial x_i} = \sum_{j=1}^{2} \frac{\partial p}{\partial g_j}
(\sum_{k=1}^{2} \frac{\partial g_j}{\partial f_k} \frac{\partial f_k}{\partial x_i}),
\space \forall i \in \lbrace 1, 2 \rbrace.\\&
\frac{\partial f_1}{\partial x_1} = \frac{\partial f_1}{\partial x_2} = 1,\\&
\frac{\partial f_2}{\partial x_1} = x_2, \frac{\partial f_2}{\partial x_2} = x_1,\\&
\frac{\partial g_1}{\partial f_1} = \frac{\partial g_1}{\partial f_2} = \frac{1}{cos^2(f_1 + f_2 + 100)},\\&
\frac{\partial g_2}{\partial f_1} = f_2, \frac{\partial g_2}{\partial f_2} = f_1,\\&
\frac{\partial y}{\partial x_1} =
\frac{\partial p}{\partial g_1} \frac{1+x_2}{cos^2(f_1 + f_2 + 100)} +
\frac{\partial p}{\partial g_2} (f_2 + f_1 x_2),\\&
\frac{\partial y}{\partial x_2} =
\frac{\partial p}{\partial g_1} \frac{1+x_1}{cos^2(f_1 + f_2 + 100)} +
\frac{\partial p}{\partial g_2} (f_2 + f_1 x_1).
\end{align*}
$

In [ ]:
import math
import pandas as pd
import torch

In [ ]:
def f1(x1, x2):
    return x1 + x2


def f2(x1, x2):
    return x1 * x2


def dg1df(x1, x2):
    return torch.cos(f1(x1, x2) + f2(x1, x2) + 100) ** -2


def dydx1(x1, x2, dpdg1, dpdg2):
    return dpdg1 * (1 + x2) * dg1df(x1, x2) + dpdg2 * (f2(x1, x2) + f1(x1, x2) * x2)


def dydx2(x1, x2, dpdg1, dpdg2):
    return dpdg1 * (1 + x1) * dg1df(x1, x2) + dpdg2 * (f2(x1, x2) + f1(x1, x2) * x1)

In [ ]:
# gradients in default dtype (float32) is too approximate
torch.set_default_dtype(torch.float64)

json_path = 'dev.json'
data_frame = pd.read_json(json_path)

x1, x2, dpdg1, dpdg2 = [torch.tensor(data_frame[i], requires_grad=True)
                        for i in ('x1', 'x2', 'dpdg1', 'dpdg2')]

g1 = torch.tan(x1 + x2 + x1 * x2 + 100)
g2 = (x1 + x2) * x1 * x2
g = torch.stack([g1, g2])
g.backward(gradient=torch.stack([dpdg1, dpdg2]))

# gradients provided by Torch
tx1, tx2 = x1.grad, x2.grad
# gradients provided analytically
ax1, ax2 = dydx1(x1, x2, dpdg1, dpdg2), dydx2(x1, x2, dpdg1, dpdg2)

ids = range(len(x1))

match = [math.isclose(tx1[i], ax1[i], rel_tol=1e-8) and
         math.isclose(tx2[i], ax2[i], rel_tol=1e-8) for i in ids]
assert not False in match, "Gradient mismatch!"

csv_path = 'dev.csv'
grads = pd.DataFrame({'id': [i + 1 for i in ids], 'dx1': tx1.numpy(), 'dx2': tx2.numpy()})
grads.to_csv(csv_path, index=False)